<a href="https://colab.research.google.com/github/pscabral/IFMASOFTEX_1/blob/main/mobilenetv2_9.00_160.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mega.py

from mega import Mega

# Crie uma instância da classe Mega
mega = Mega()

# Cole o link completo do Mega.nz (incluindo a chave)
url = "https://mega.nz/file/xutxkLoY#5ycY65KDsxtwEobjIycZoITipxk92yA3yQEgkImDYgM"

# Use o link completo para fazer o download
mega.download_url(url)

!unzip GroceryStoreDataset_.zip

In [ ]:
import os
import cv2
from tqdm import tqdm
import numpy as np

def perform_augmentation(image):
    # Exemplo: Rotação aleatória entre -10 e 10 graus
    angle = np.random.uniform(low=-10, high=10)
    rows, cols, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    augmented_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))
    return augmented_image

def apply_data_augmentation(input_dir, augmentation_factor):
    for subdir, _, files in os.walk(input_dir):
        # Filtrar apenas arquivos de imagem
        image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]

        print(f"Encontrados {len(image_files)} arquivos de imagem em {subdir}")

        for image_file in tqdm(image_files, desc=f"Aumentando dados em {subdir}"):
            image_path = os.path.join(subdir, image_file)

            # Verificar se a imagem foi carregada corretamente
            original_image = cv2.imread(image_path)
            if original_image is None:
                print(f"Erro ao carregar a imagem: {image_path}")
                continue

            # Corrigir a conversão do augmentation_factor para um valor inteiro
            num_augmented_images = int(len(image_files) * augmentation_factor)

            for i in range(num_augmented_images):
                # Aplicar técnicas de aumento de dados (por exemplo, espelhamento, rotação, zoom)
                augmented_image = perform_augmentation(original_image)

                # Salvar a imagem aumentada no mesmo subdiretório
                output_file = os.path.join(subdir, f"{image_file.split('.')[0]}_{i}.jpg")
                cv2.imwrite(output_file, augmented_image)

# Diretórios de entrada para cada conjunto de dados
datasets = [
    "/content/GroceryStoreDataset_/test",
    "/content/GroceryStoreDataset_/train",
    "/content/GroceryStoreDataset_/val",
]

# Aplicar aumento de dados para cada conjunto de dados
for input_dir in datasets:
    apply_data_augmentation(input_dir, 0.3)  # Ajuste o fator de aumento conforme necessário


In [19]:
import numpy as np
import pandas as pd
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import cv2

from sklearn.metrics import classification_report, confusion_matrix

import os

import warnings
warnings.filterwarnings('ignore')

In [20]:
train_dir = "../content/GroceryStoreDataset_/train"
train_class=os.listdir(train_dir)
print(train_class)

val_dir = "../content/GroceryStoreDataset_/val"
val_class=os.listdir(val_dir)
print(val_class)

test_dir = "../content/GroceryStoreDataset_/test"
test_class=os.listdir(test_dir)
print(test_class)

['Zucchini', 'God-Morgon-Red-Grapefruit-Juice', 'Sweet-Potato', 'Floury-Potato', 'Oatly-Natural-Oatghurt', 'Garant-Ecological-Standard-Milk', 'Papaya', 'Cantaloupe-Melon', 'Apple-Granny-Smith', 'Cucumber', 'Plum', 'Alpro-Fresh-Soy-Milk', 'Pear-Conference', 'Satsumas', 'Nectarine', 'Peach', 'Orange-Bell-Pepper', 'Apple-Golden-Delicious', 'God-Morgon-Apple-Juice', 'Mango', 'Red-Beet', 'Bravo-Apple-Juice', 'Leek', 'Yellow-Bell-Pepper', 'Tropicana-Apple-Juice', 'Arla-Ecological-Sour-Cream', 'Arla-Natural-Yoghurt', 'Arla-Medium-Fat-Milk', 'Banana', 'Solid-Potato', 'Asparagus', 'Tropicana-Golden-Grapefruit-Juice', 'God-Morgon-Orange-Juice', 'Yellow-Onion', 'Alpro-Blueberry-Soyghurt', 'Avocado', 'Pear-Anjou', 'Apple-Royal-Gala', 'Watermelon-Melon', 'God-Morgon-Orange-Red-Grapefruit-Juice', 'Beef-Tomato', 'Orange', 'Aubergine', 'Arla-Sour-Cream', 'Brown-Cap-Mushroom', 'Tropicana-Juice-Smooth-Juice', 'Arla-Mild-Vanilla-Yoghurt', 'Yoggi-Vanilla-Yoghurt', 'Valio-Vanilla-Yoghurt', 'Yoggi-Strawberr

In [ ]:
x=[] # array value of the images
y=[] # the labels of the images

for folder in os.listdir(train_dir):
    image_list=os.listdir(train_dir+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(train_dir+"/"+folder+"/"+img_name,target_size=(224,224))
        img=image.img_to_array(img)

        # Transfer Learning: Preprocess of MobileNetV2 model to our images before passing it to MobileNetV2
        img=preprocess_input(img)

        x.append(img) # appending image array
        y.append(train_class.index(folder)) # appending class index to the array

print("Preparing Training Dataset Completed.")

In [ ]:
val_images=[]
val_images_Original=[]
val_image_label=[]

for folder in os.listdir(val_dir):
    image_list=os.listdir(val_dir+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(val_dir+"/"+folder+"/"+img_name,target_size=(224,224))

        # Converting to arrarys
        img=image.img_to_array(img)

        # Saving original images, will be used just for display at the end
        val_images_Original.append(img.copy())
        img=preprocess_input(img)

        # Appending arrays
        val_images.append(img)
        val_image_label.append(val_class.index(folder))

print("Preparing Validation Dataset Completed.")

In [ ]:
test_images=[]
test_images_Original=[]
test_image_label=[]

for folder in os.listdir(test_dir):
    image_list=os.listdir(test_dir+"/"+folder)
    for img_name in image_list:
        # Loading images
        img=image.load_img(test_dir+"/"+folder+"/"+img_name,target_size=(224,224))

        # Converting to arrarys
        img=image.img_to_array(img)

        # Saving original images, will be used just for display at the end
        test_images_Original.append(img.copy())
        img=preprocess_input(img)

        # Appending arrays
        test_images.append(img)
        test_image_label.append(test_class.index(folder))

print("Preparing Test Dataset Completed.")